In [42]:
# 턴어라운드 게임

In [ ]:
import csv
import random
import numpy as np
from tensorflow.python.framework import ops
import tensorflow as tf
ops.reset_default_graph()

In [43]:
response = 6
batch_size = 50
symmetry = ['rotate180', 'rotate90', 'rotate270', 'flip_v', 'flip_h']

In [44]:
def print_board(board):
    symbols = ['O', ' ', 'X']
    board_plus1 = [int(x) + 1 for x in board]
    print(' ' + symbols[board_plus1[0]] + ' l '
         + symbols[board_plus1[1]]
         +' l '+ symbols[board_plus1[2]] )
    print('-----------------')
    print(' ' + symbols[board_plus1[3]] + ' l '
         + symbols[board_plus1[4]]
         + ' l ' + symbols[board_plus1[5]])
    print('-----------------')
    print(' ' + symbols[board_plus1[6]] + ' l '
         + symbols[board_plus1[7]]
         + ' l ' + symbols[board_plus1[8]])

In [45]:
def symmetry(board, response, transformation):
    if transformation == 'rotate180':
        new_response = 8 - response # 말을 놓을 자리
        return(board[::-1], new_response) # 역순으로
    elif transformation == 'rotate90':
        new_response = [6,3,0,7,4,1,8,5,2].index(response)
        tuple_board = list(zip(*[board[6:9], board[3:6], board[0:3]]))
        return([value for item in tuple_board for value in item], new_response)

In [46]:
# 다양한 경우를 충족하기 위해 변화
def get_symmetry(board, response, transformation):
    if transformation == 'rotate180':
        new_response = 8 - response
        return(board[::-1], new_response)
    elif transformation == 'rotate90':
        new_response = [6,3,0,7,4,1,8,5,2].index(response)
        tuple_board = list(zip(*[board[6:9], board[3:6], board[0:3]]))
        return([value for item in tuple_board for value in item], new_response)
    elif transformation == 'rotate270':
        new_response = [2,5,8,1,4,7,0,3,6].index(response)
        tuple_board = list(zip(*[board[0:3], board[3:6], board[6:9]]))[::-1]
        return([value for item in tuple_board for value in item], new_response)
    elif transformation == 'flip_v':
        new_response = [6,7,8,3,4,5,0,1,2].index(response)
        return(board[6:9] + board[3:6]+board[0:3], new_response)
    else:
        raise ValueError('해당하는 경우가 없음')

In [47]:
# 파일로부터 말 규칙 로딩
def get_moves_from_csv(csv_file):
    moves = []
    with open(csv_file, 'rt') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            moves.append(([int(x) for x in row[0:9]], int(row[9])))
    return moves

In [48]:
# 정해진 말규칙을 다양하게 변화하도록 훈련용 데이터 확장
def get_rand_move(moves, n=1, rand_transforms=2):
    (board, response) = random.choice(moves)
    possible_transforms = ['rotate90', 'rotate180', 'rotate270', 'flip_v', 'flip_h']
    for i in range(rand_transforms):
        random_transform = random.choice(possible_transforms)
        (board, response) = get_symmetry(board, response, random_transform)
    return(board, response)

In [49]:
moves = get_moves_from_csv('tictactoe_moves.csv')
train_length = 500
train_set = []
for t in range(train_length):
    train_set.append(get_rand_move(moves))

ValueError: 해당하는 경우가 없음

In [50]:
def init_weights(shape):
    return(tf.Variable(tf.random_normal(shape)))

In [51]:
def model(X, A1, A2, bias1, bias2):
    layer1 = tf.nn.sigmoid(tf.add(tf.matmul(X, A1), bias1))
    layer2 = tf.add(tf.matmul(layer1, A2), bias2)
    return(layer2)

In [52]:
X = tf.placeholder(dtype=tf.float32, shape=[None, 9])
Y = tf.placeholder(dtype=tf.int32, shape=[None])
A1 = init_weights([9,81])
bias1 = init_weights([81])
A2 = init_weights([81, 9])
bias2 = init_weights([9])
model_output = model(X, A1, A2, bias1, bias2)

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model_output, labels=Y))
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)
prediction = tf.argmax(model_output, 1)
sess = tf.Session()
init =tf.global_variables_initializer()
sess.run(init)

In [56]:
loss_vec = []
for i in range(10000):
    rand_indices = np.random.choice(range(len(train_set)), batch_size, replace=False)
    batch_data = [train_set[i] for i in rand_indices]
    x_input = [x[0] for x in batch_data]
    y_target = np.array([y[1] for y in batch_data])
    sess.run(train_step, feed_dict={X: x_input, Y: y_target})
    temp_loss = sess.run(loss, feed_dict={X: x_input, Y: y_target})
    loss_vec.append(temp_loss)
    if i % 500 == 0:
        print('iteration' + str(i) + 'Loss:' + str(temp_loss))

ValueError: Cannot take a larger sample than population when 'replace=False'

In [54]:
test_board = [-1, 0, 0, 1, -1, -1, 0, 0, 1]
train_set = [x for x in train_set if x[0] != test_board]

In [55]:
# 게임 승패 결정
def check(board):
    wins = [[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]
    for i in range(len(wins)):
        if board[wins[i][0]] == board[wins[i][1]] == board[wins[i][2]] == 1.:
            return(1)
        elif board[wins[i][0]]==board[wins[i][1]] == board[wins[i][2]] == -1.:
            return(-1)
    return(0)

In [29]:
game_tracker = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
win_logical = False
num_moves = 0
while not win_logical:
    player_index = input("이동하고 싶은 인덱스를 입력하시오 (0-8):")
    num_moves += 1
    game_tracker[int(player_index)] = 1.
    [potential_moves] = sess.run(model_output, feed_dict={X:game_tracker})
    allowed_moves = [ix for ix, x in enumerate(game_tracker) if x==0.0]
    model_move = np.argmax([x if ix in allowed_moves else -999.0
                            for ix, x in enumerate(potential_moves)])
    game_tracker[int(model_move)] = -1.
    print('모델이 이동하였습니다.')
    print_board(game_tracker) # GUI 방식
    if check(game_tracker) == 1 or num_moves>=20:
        print("게임 종료! 승리하셨습니다. ")
        win_logical = True
    elif check(game_tracker) == -1:
        print("게임 종료! 게임에 지셨습니다. ")
        win_logical = True

이동하고 싶은 인덱스를 입력하시오 (0-8):4


ValueError: Cannot feed value of shape (9,) for Tensor 'Placeholder:0', which has shape '(?, 9)'